<a href="https://colab.research.google.com/github/psakore/IMCFeedArbitration/blob/main/imc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install scapy

In [101]:
import sys
from scapy.all import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

MAX_TIME = 9999999999999999

In [62]:
def process_packet(packet, data_dict):
    # Extract relevant information from the packet (symbol, seqno, timestamp)
    payload = packet.load.decode("utf-8")
    symbol = re.search(r"Symbol: (\w+)", payload).group(1)
    seqno = int(re.search(r"Seqno: (\d+)", payload).group(1))
    timestamp = int(packet.time * 1000000) # converting to int
    source_ip = packet[IP].src  # Get the source IP address

    # Add the packet information to the data dictionary
    key = "%s_%d" % (symbol, seqno)
    if key not in data_dict:
      data_dict[key] = {}
    data_dict[key][source_ip] = timestamp

In [56]:
def calculate_metrics(data_dict, num_best):
    publishers_tally = {}
    for symbol, symbol_data in data_dict.items():
        data_dict[symbol]['fastest_pub'] = min(symbol_data['publishers'], key=symbol_data['publishers'].get)


In [125]:
def main(pcap_file, num_best):
    # Initialize a data dictionary to store packet information
    data_dict = {}

    # Iterate through packets, process them, and calculate metrics
    for packet in PcapReader(pcap_file):
        if packet.haslayer(UDP):
            process_packet(packet, data_dict)

    #calculate_metrics(data_dict, num_best)
    print(data_dict)

    df = pd.DataFrame(data_dict).T
    df = df.reindex(sorted(df.columns), axis=1)
    df.index.name = 'Symbol_Seqno'
    df.fillna(MAX_TIME, inplace=True)
    df = df.astype(int)

    df['Fastest_Publisher'] = df.idxmin(axis=1)


    top_values = df['Fastest_Publisher'].value_counts().head(num_best).index.tolist()

    # Filter the DataFrame to include only columns in top_values
    filtered_df = df[top_values]
    min_time_df = filtered_df.min(axis=1)

    print(min_time_df)

main('/content/drive/MyDrive/IMC Assignment/feed_arbitrage.pcap', 2)

{'APPL_0': {'10.10.10.4': 1473411962167382, '10.10.10.1': 1473411962413131, '10.10.10.3': 1473411962542158, '10.10.10.2': 1473411962555594}, 'APPL_1': {'10.10.10.4': 1473411963112848, '10.10.10.1': 1473411963499451, '10.10.10.3': 1473411963590191, '10.10.10.2': 1473411963599473}, 'APPL_2': {'10.10.10.2': 1473411964328186, '10.10.10.1': 1473411964411783, '10.10.10.3': 1473411964527478, '10.10.10.4': 1473411964543972}, 'APPL_3': {'10.10.10.4': 1473411965345962, '10.10.10.1': 1473411965364084, '10.10.10.3': 1473411965591194, '10.10.10.2': 1473411965600693}, 'APPL_4': {'10.10.10.4': 1473411966149519, '10.10.10.3': 1473411966320074, '10.10.10.2': 1473411966586622, '10.10.10.1': 1473411966589680}, 'APPL_5': {'10.10.10.4': 1473411967170404, '10.10.10.3': 1473411967354888, '10.10.10.2': 1473411967534771, '10.10.10.1': 1473411967550518}, 'APPL_6': {'10.10.10.4': 1473411968335903, '10.10.10.2': 1473411968413411, '10.10.10.3': 1473411968516188, '10.10.10.1': 1473411968588516}, 'APPL_7': {'10.10.1